## Turtle Trading Strategy 

Time Period: 2012-01-01 to 2016-03-10, ￥1000000

In [ ]:
import jqdatasdk
import pandas as pd
import warnings
import numpy as np

In [ ]:
def initialize(context):
    set_params()        #1.Set strategy parameters
    set_variables()     #2.Set intermediate variable
    set_backtest()      #3.Set the backtest conditions

In [ ]:
# 1.Set strategy parameters
def set_params():
    g.security = '000063.XSHE'
    # trailing date of market entry for system 1
    g.short_in_date = 20
    # trailing date of market entry for system 2
    g.long_in_date = 55
    # system 1 exiting market trailing date
    g.short_out_date = 10
    # system 2 exiting market trailing date
    g.long_out_date = 20
    # g.dollars_per_share是: the amount of change in the total price of a lot of shares per minimum unit of movement of the underlying stock.
    # In China, the minimum change is 0.01 yuan, so it is 0.01×100=1
    g.dollars_per_share = 1
    # The maximum loss rate that can be borne
    g.loss = 0.1
    # If the maximum loss rate is exceeded, the adjustment rate is:
    g.adjust = 0.8
    # Calculate the number of days for N
    g.number_days = 20
    # Maximum allowable unit
    g.unit_limit = 4
    # Proportion of the amount allocated in System 1 to the total amount
    g.ratio = 0.5

In [ ]:
# 2.Set intermediate variable
def set_variables():
    # Initial unit
    g.unit = 1000
    # A list storing info of N
    g.N = []
    # Record the number of days for this trading system
    g.days = 0
    # The breakout price for system 1
    g.break_price1 = 0
    # The breakout price of system 2
    g.break_price2 = 0
    # Number of positions built in system 1
    g.sys1 = 0
    # Number of positions built in system 2
    g.sys2 = 0
    # System 1 executes the command and system 2 does not
    g.system1 = 'False'


In [ ]:
# 3.Set the backtest conditions
def set_backtest():
    # As a benchmark to judge the quality of strategies and a series of risk-value calculations
    set_benchmark(g.security)
    set_option('use_real_price',True) #Trade at real prices
    log.set_level('order','error') # Set the error level

#Things to do every day before the market opens
def before_trading_start(context):
    set_slip_fee(context) 


In [ ]:
# 4 
# Set sliding points and handling fees according to different time periods
def set_slip_fee(context):
    # Set the slide point to 0
    set_slippage(FixedSlippage(0)) 
    # Fees are set according to different time periods
    dt=context.current_dt
    
    if dt>datetime.datetime(2013,1, 1):
        set_commission(PerTrade(buy_cost=0.0003, sell_cost=0.0013, min_cost=5)) 
        
    elif dt>datetime.datetime(2011,1, 1):
        set_commission(PerTrade(buy_cost=0.001, sell_cost=0.002, min_cost=5))
            
    elif dt>datetime.datetime(2009,1, 1):
        set_commission(PerTrade(buy_cost=0.002, sell_cost=0.003, min_cost=5))
                
    else:
        set_commission(PerTrade(buy_cost=0.003, sell_cost=0.004, min_cost=5))

In [ ]:
# Minute back test
def handle_data(context, data):
    dt = context.current_dt # current date
    current_price = data[g.security].price # Current price: Daily 9:30 a.m
    if dt.hour==9 and dt.minute==30:
        g.days += 1
        calculate_N() # Calculate the value of N
    if g.days > g.number_days:
        # The total value of stocks and cash currently held
        value = context.portfolio.portfolio_value
        # Cash to spend
        cash = context.portfolio.cash 
        if g.sys1 == 0 and g.sys2 == 0:
            # If the loss rate is greater than G.OSS, the cash and total value held are adjusted (reduced)
            if value < (1-g.loss)*context.portfolio.starting_cash:
                cash *= g.adjust
                value *= g.adjust
                
        # Calculate the fluctuating price of the US dollar
        dollar_volatility = g.dollars_per_share*(g.N)[-1]
        # According to this strategy, the units of the transaction are calculated
        g.unit = value*0.01/dollar_volatility

In [ ]:
# Operation of system 1
        g.system1 = 'True'
        if g.sys1==0:
            market_in(current_price, (g.ratio)*cash, g.short_in_date)
        else:
            stop_loss(current_price)
            market_add(current_price, (g.ratio)*cash, g.short_in_date)
            market_out(current_price, g.short_out_date)  


In [ ]:
# Operation of system 2
        g.system1 = 'False'
        if g.sys2==0:
            market_in(current_price, (1-g.ratio)*cash, g.long_in_date)
        else:
            stop_loss(current_price)
            market_add(current_price, (1-g.ratio)*cash, g.long_in_date)
            market_out(current_price, g.long_out_date)   
        

In [ ]:
# 5
# Calculate the current value of N
# Input：none
# Output：The updated list-list type for the values of N
def calculate_N():
    # If the trading days are less than or equal to 20 days
    if g.days <= g.number_days:
        price = attribute_history(g.security, g.days, '1d',('high','low','pre_close'))
        lst = []
        for i in range(0, g.days):
            h_l = price['high'][i]-price['low'][i]
            h_c = price['high'][i]-price['pre_close'][i-1]
            c_l = price['pre_close'][i-1]-price['low'][i]
            # Calcualate True Range
            True_Range = max(h_l, h_c, c_l)
            lst.append(True_Range)
        # Calculate the True_Range average for the days before g.Ays (20 or less), i.e., the current value of N:
        current_N = np.mean(np.array(lst))
        (g.N).append(current_N)
        
    # If the trading days exceed 20 days
    else:
        price = attribute_history(g.security, 2, '1d',('high','low','close'))
        h_l = price['high'][-1]-price['low'][-1]
        h_c = price['high'][-1]-price['close'][-2]
        c_l = price['close'][-2]-price['low'][-1]
        # Calculate the True Range
        True_Range = max(h_l, h_c, c_l)
        # The True_Range average of the preceding g.umber_days (greater than 20) days, i.e., the current value of N:
        # (19/20) --> 1
        current_N = (True_Range + (g.number_days-1)*(g.N)[-1])/g.number_days
        (g.N).append(current_N)


In [ ]:
#6
# Enter the market: Decide whether system 1 and system 2 should enter the market, update the breakthrough price of system 1 and system 2
# Turtle divides all funds into 2 parts: some funds are executed according to System 1, and some funds are executed according to System 2
# Enter: Current price -float, cash -float, days -int
# output：none
def market_in(current_price, cash, in_date):
    # Get the price for the past "in_date" days
    price = attribute_history(g.security, in_date, '1d', ('close'))
    # Build position if current price is higher than highest in past
    if current_price > max(price['close']):
        # Calculate the number of shares you can buy
        num_of_shares = cash/current_price
        print("The number of shares available for purchase：%s" % num_of_shares)
        if num_of_shares >= g.unit:
            print ("Stock price at 9:30am today：%s" % current_price)
            print ("The highest stock price in the last 20 days：%s" % max(price['close']))
            if g.system1 == 'True':
                if g.sys1 < int(g.unit_limit*g.unit):
                    print ("System 1 purchase：%s" % g.unit)
                    order(g.security, int(g.unit))
                    g.sys1 += int(g.unit)
                    g.break_price1 = current_price
            if g.system1 == 'False':
                if g.sys2 < int(g.unit_limit*g.unit):
                    print ("System 2 purchase：%s" % g.unit)
                    order(g.security, int(g.unit))
                    g.sys2 += int(g.unit)
                    g.break_price2 = current_price

In [ ]:
#7
# Adding positions function
# Enter: Current price -float, cash -float, days -int
# output：none
def market_add(current_price, cash, in_date):
    if g.system1 == 'True':
        break_price=g.break_price1
    if g.system1 == 'False':
        break_price=g.break_price2
    # Every time it goes up by 0.5N, add one unit
    if current_price >= break_price + 0.5*(g.N)[-1]: 
        num_of_shares = cash/current_price
        # Adding positions 
        if num_of_shares >= g.unit: 
            print ("Adding positions：%s" % g.unit)
            print ("Position in system 1：%s" % g.sys1)
            print ("Position in system 2：%s" % g.sys2)
            print ("Atock price at 9:30am today：%s" % current_price)
            upper_bound = break_price + 0.5*(g.N)[-1]
            print ("Today's intraday stock price above the line：%s" % upper_bound)
       
            if g.system1 == 'True':
                if g.sys1 < int(g.unit_limit*g.unit):
                    order(g.security, int(g.unit))
                    g.sys1 += int(g.unit)
                    g.break_price1 = current_price
                    print("System 1 goes online above the breakout price：%s" % g.break_price1)
            if g.system1 == 'False':
                if g.sys2 < int(g.unit_limit*g.unit):
                    order(g.security, int(g.unit))
                    g.sys2 += int(g.unit)
                    g.break_price2 = current_price
                    print("System 2 goes online above the breakout price：%s" % g.break_price1)

In [ ]:
#8
# Stock market exit
# Enter: current price-float, days -int
# output：none
def market_out(current_price, out_date):
    # Function for leaving the market
    price = attribute_history(g.security, out_date, '1d', ('close'))
    # If the current price is lower than the minimum closing price on the previous out_date day, sell all positions
    if current_price < min(price['close']):
        print ("Exit Market")
        print ("Stock price at 9:30am today：%s" % current_price)
        print ("The lowest stock price in the last 20 days：%s" % min(price['close']))
        if g.system1 == 'True':
            if g.sys1>0:
                order(g.security, -g.sys1)
                g.sys1 = 0
        if g.system1 == 'False':
            if g.sys2>0:
                order(g.security, -g.sys2)
                g.sys2 = 0

In [ ]:
# 9
# Stop loss function
# Enter: Current price -float
# output：none
def stop_loss(current_price):
    # If the loss is greater than 2N, sell the stock
    if g.system1 == 'True':
        break_price = g.break_price1
    if g.system1 == 'False':
        break_price = g.break_price2
    # If the price has decreased by 2N, then clear all position
    if current_price < (break_price - 2*(g.N)[-1]):
        print ("Stop loss")
        print ("Stock price at 9:30am today：%s" % current_price)
        lower_bound = break_price - 2*(g.N)[-1]
        if g.system1 == True:
            print ("System 1 goes offline below the breakout price：%s" % lower_bound)
            order(g.security, -g.sys1)
            g.sys1 = 0  
        if g.system1 == False:
            print ("System two is off line below the breakout price：%s" % lower_bound)
            order(g.security, -g.sys2)
            g.sys2 = 0
